# Imports

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
from tqdm import tqdm

sns.set_theme()

# Data Loading

In [42]:

def load_all_experiments(experiment_folder="./experiments"):
    """
    Given a folder with a number of experiments, where each experiment has one pickled dataframe, load and return all
    dataframes.

    """
    # TODO: Read from configs, too, and insert these values as metadata.
    experiment_folder = Path(experiment_folder)
    if not experiment_folder.is_dir():
        raise ValueError(f"Not a valid folder: {experiment_folder}")

    all_frames = {}
    for d in tqdm(Path(experiment_folder).iterdir(), "Scanning all experiments"):
        if d.is_dir():
            gen_pkl = d / "generations.pkl"
            if gen_pkl.is_file():
                df = pd.read_pickle(gen_pkl)
                df["experiment"] = d.name
                df.reset_index(inplace=True)
                df.set_index(["experiment", "generation"], inplace=True)
                all_frames[d.name] = df
    return all_frames


In [85]:
experiment_map = load_all_experiments(".")
print(f"Found {len(experiment_map)} experiments.")

Scanning all experiments: 15it [00:00, 47.72it/s]

Found 3 experiments.


# Collate All Experiments Together

In [86]:
experiments = pd.concat(experiment_map.values())

# Analysis

In [87]:
experiments.groupby("experiment").max("fitness.max")["fitness.max"].sort_values(ascending=False)

# TODO: Get last value from each group.

experiment
walker-results   -18.351097
results          -51.081411
configs          -69.583416
Name: fitness.max, dtype: float64

In [88]:
experiments.groupby("experiment").max("fitness.mean")["fitness.mean"].sort_values(ascending=False)

experiment
walker-results   -109.983495
results          -111.885640
configs          -122.726930
Name: fitness.mean, dtype: float64

# Scratch Area

In [84]:
"1   3".split(" ")

['1', '', '', '3']